Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [4]:
#first follow the instructions in the README.md file under Local Installation
!pip3 install -r requirements.txt
# !python3 setup.py install

  Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
  Using cached rotary_embedding_torch-0.6.1-py3-none-any.whl (5.3 kB)
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached tokenizers-0.19.1-cp310-none-win_amd64.whl (2.2 MB)
  Using cached inflect-7.2.1-py3-none-any.whl (34 kB)
  Using cached progressbar-2.5.tar.gz (10 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached einops-0.4.1-py3-none-any.whl (28 kB)
  Using cached Unidecode-1.3.8-py3-none-any.whl (235 kB)
  Using cached scipy-1.13.0-cp310-cp310-win_amd64.whl (46.2 MB)
  Using cached librosa-0.9.1-py3-none-any.whl (213 kB)
  Using cached ffmpeg-1.4.tar.gz (5.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
  Using cached numba-0.59.1-cp310-cp310-win_amd64.whl (2.7 MB)
  Using cached torchaudio-2.3.0-cp310-cp310-win_

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [9 lines of output]
      [WARNING] Unable to import torch, pre-compiling ops will be disabled. Please visit https://pytorch.org/ to see how to properly install torch on your system.
       [WARNING]  unable to import torch, please install it if you want to pre-compile any deepspeed ops.
      DS_BUILD_OPS=1
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\priyatosh.anand\AppData\Local\Temp\pip-install-k_p6vqgc\deepspeed_3e9c992708194b7c9c188a6e01025b71\setup.py", line 122, in <module>
          assert torch_available, "Unable to pre-compile ops without torch installed. Please install torch before attempting to pre-compile ops."
      AssertionError: Unable to pre-compile ops without torch installed. Please install torch before attempting to pre-comp

In [1]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
# tts = TextToSpeech()
# If you want to use deepspeed the pass use_deepspeed=True nearly 2x faster than normal
tts = TextToSpeech()

c:\Users\priyatosh.anand\AppData\Local\anaconda3\envs\tortoise\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\priyatosh.anand\AppData\Local\anaconda3\envs\tortoise\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [4]:
# This is the text that will be spoken.
text = "Supreme court order on interim bail of Delhi chief minister Arvind kejriwal on May 10"

# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [5]:
# Tortoise will attempt to mimic voices you provide. It comes pre-packaged
# with some voices you might recognize.

# Let's list all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
# !ls tortoise/voices

!dir tortoise\voices



IPython.display.Audio('tortoise/voices/Indian_Palki_Sharma')

 Volume in drive C has no label.
 Volume Serial Number is A636-7FFA

 Directory of c:\Users\priyatosh.anand\Desktop\project\tortoise-tts\tortoise\voices

05/13/2024  04:59 PM    <DIR>          .
05/09/2024  09:05 PM    <DIR>          ..
05/09/2024  07:19 PM    <DIR>          angie
05/09/2024  07:19 PM    <DIR>          applejack
05/09/2024  07:19 PM    <DIR>          cond_latent_example
05/09/2024  07:19 PM    <DIR>          daniel
05/09/2024  07:19 PM    <DIR>          deniro
05/09/2024  07:19 PM    <DIR>          emma
05/09/2024  07:19 PM    <DIR>          freeman
05/09/2024  07:19 PM    <DIR>          geralt
05/09/2024  07:19 PM    <DIR>          halle
05/12/2024  05:16 PM    <DIR>          indian_news_voice
05/13/2024  04:59 PM    <DIR>          Indian_Palki_Sharma
05/09/2024  07:19 PM    <DIR>          jlaw
05/09/2024  07:19 PM    <DIR>          lj
05/09/2024  07:19 PM    <DIR>          mol
05/09/2024  07:19 PM    <DIR>          myself
05/09/2024  07:19 PM    <DIR>          pat
05

PermissionError: [Errno 13] Permission denied: 'tortoise/voices/Indian_Palki_Sharma'

In [12]:
# Pick one of the voices from the output above
voice = 'indian_news_voice'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

Generating autoregressive samples..


100%|██████████| 256/256 [2:02:27<00:00, 28.70s/it]  


Computing best candidates using CLVP


100%|██████████| 256/256 [12:59<00:00,  3.05s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [30:40<00:00,  4.60s/it]


In [ ]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.", 
                          voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

In [ ]:
del tts  # Will break other cells, but necessary to conserve RAM if you want to run this cell.

# Tortoise comes with some scripts that does a lot of the lifting for you. For example,
# read.py will read a text file for you.
!python3 tortoise/read.py --voice=train_atkins --textfile=tortoise/data/riding_hood.txt --preset=ultra_fast --output_path=.

IPython.display.Audio('train_atkins/combined.wav')
# This will take awhile..